In [ ]:
from category_encoders import TargetEncoder
import math
from sklearn.preprocessing import Normalizer,normalize
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# target = 'Rainfall', RainTomorrow'
encoder1 = TargetEncoder()
encoder2 = TargetEncoder()
encoder3 = TargetEncoder()
encoder4 = TargetEncoder()

# trainColumns = list(set(train.columns)-set(['Rainfall', 'RainTomorrow']))
nor = Normalizer()
target = MinMaxScaler()

def transform(x):
   return  np.exp(-x/2)# np.exp(-(x-mean)/variance)#np.exp(-x/2) #np.exp(-x/2)
def inverse(x):
   return -2*np.log(x)# -variance* np.log(x)+mean #-2*np.log(x) #-2*np.log(x)

In [ ]:
def convert_date(data):
    #There don't seem to be any error in dates so parsing values into datetime
    data['Date']= pd.to_datetime(data["Date"])
    #Creating a collumn of year
    data['year'] = 2022-data.Date.dt.year

    # function to encode datetime into cyclic parameters. 
    #As I am planning to use this data in a neural network I prefer the months and days in a cyclic continuous feature. 

    def encode(data, col, max_val):
        data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
        data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
        return data

    data['month'] = data.Date.dt.month
    data = encode(data, 'month', 12)

    data['day'] = data.Date.dt.day
    data = encode(data, 'day', 31)

#     data.head()
    return data


In [ ]:
def prepare_train(train):
    train.drop(columns=['Unnamed: 0'],inplace = True)
    train['RainTomorrow'] = np.where(train["RainTomorrow"] == "Yes", 1, 0)
    train['RainTomorrow'] = train['RainTomorrow'].astype(np.int8)
        
    train[["year", "month", "day"]] = train["Date"].str.split("-", expand = True)
    train.drop(inplace = True,columns=['Date'])

    train['day'] = train['day'].astype(np.int16)
    train['month'] = train['month'].astype(np.int16)
    train['year'] = 2022 - train['year'].astype(np.int16)
#     train = convert_date(train)
#     train.drop(inplace = True,columns=['Date'])
    
    train['Location'] = encoder1.fit_transform(train['Location'], train[['Rainfall']])
    train['WindDir9am'] = encoder2.fit_transform(train['WindDir9am'], train[['Rainfall']])
    train['WindDir3pm'] = encoder3.fit_transform(train['WindDir3pm'], train[['Rainfall']])
    train['WindGustDir'] = encoder4.fit_transform(train['WindGustDir'], train[['Rainfall']])
    
    train['min_max_temp_avg'] = train[['MinTemp','MaxTemp']].mean(axis=1)
    train['pressure_3_9_avg'] = train[['Pressure9am','Pressure3pm']].mean(axis=1)
    train['temp_3_9_avg'] = train[['Temp9am','Temp3pm']].mean(axis=1)
   
    train.drop(columns=['MinTemp','MaxTemp','Pressure9am','Pressure3pm','Temp9am','Temp3pm'],inplace = True)
    trainColumns = list(set(train.columns)-set(['Rainfall', 'RainTomorrow']))
#     'year','month','day'
    train[trainColumns] = nor.fit_transform(train[trainColumns])
#     train['Rainfall'] = (train['Rainfall']-2.9) **(1/3)
    train['Rainfall'] = train['Rainfall'].apply(transform)
    train['Rainfall'] = target.fit_transform(train[['Rainfall']])
    return train

In [ ]:
def prepare_validation(train):
    train.drop(columns=['Unnamed: 0'],inplace = True)
    train['RainTomorrow'] = np.where(train["RainTomorrow"] == "Yes", 1, 0)
    train['RainTomorrow'] = train['RainTomorrow'].astype(np.int8)
        
    train[["year", "month", "day"]] = train["Date"].str.split("-", expand = True)
    train.drop(inplace = True,columns=['Date'])

    train['day'] = train['day'].astype(np.int16)
    train['month'] = train['month'].astype(np.int16)
    train['year'] = 2022 - train['year'].astype(np.int16)
    ## create new features based on date
#     train = convert_date(train)
#     train.drop(inplace = True,columns=['Date'])
    
    train['Location'] = encoder1.transform(train['Location'])
    train['WindDir9am'] = encoder2.transform(train['WindDir9am'])
    train['WindDir3pm'] = encoder3.transform(train['WindDir3pm'])
    train['WindGustDir'] = encoder4.transform(train['WindGustDir'])
    
    train['min_max_temp_avg'] = train[['MinTemp','MaxTemp']].mean(axis=1)
    train['pressure_3_9_avg'] = train[['Pressure9am','Pressure3pm']].mean(axis=1)
    train['temp_3_9_avg'] = train[['Temp9am','Temp3pm']].mean(axis=1)
   
    train.drop(columns=['MinTemp','MaxTemp','Pressure9am','Pressure3pm','Temp9am','Temp3pm'],inplace = True)
    trainColumns = list(set(train.columns)-set(['Rainfall', 'RainTomorrow']))
#     'year','month','day'
    train[trainColumns] = nor.transform(train[trainColumns])
#     train['Rainfall'] = (train['Rainfall']-2.9) **(1/3)
    train['Rainfall'] = train['Rainfall'].apply(transform)
    train['Rainfall'] = target.transform(train[['Rainfall']])
    return train

In [ ]:
def prepare_test(test):
    test.drop(columns=['Unnamed: 0'],inplace = True)
    
    test[["year", "month", "day"]] = test["Date"].str.split("-", expand = True)
    test.drop(inplace = True,columns=['Date'])

    test['day'] = test['day'].astype(np.int16)
    test['month'] = test['month'].astype(np.int16)
    test['year'] = 2022 - test['year'].astype(np.int16)
#     test = convert_date(test)
#     test.drop(inplace = True,columns=['Date'])
    
    test['Location'] = encoder1.transform(test['Location'])
    test['WindDir9am'] = encoder2.transform(test['WindDir9am'])
    test['WindDir3pm'] = encoder3.transform(test['WindDir3pm'])
    test['WindGustDir'] = encoder4.transform(test['WindGustDir'])
        
    test['min_max_temp_avg'] = test[['MinTemp','MaxTemp']].mean(axis=1)
    test['pressure_3_9_avg'] = test[['Pressure9am','Pressure3pm']].mean(axis=1)
    test['temp_3_9_avg'] = test[['Temp9am','Temp3pm']].mean(axis=1)

    test.drop(columns=['MinTemp','MaxTemp','Pressure9am','Pressure3pm','Temp9am','Temp3pm'],inplace = True)
    test = pd.DataFrame(nor.transform(test),columns=list(test.columns),index=None)
    return test

In [ ]:
from  sklearn.model_selection import train_test_split

train,validation = train_test_split(pd.read_csv('/kaggle/input/tah-rain-predection/train.csv'),test_size=0.2)
validation.to_csv('validation.csv',index=None)
train.to_csv('train.csv',index=None)
validation_size = len(validation)

In [ ]:
# # # import matplotlib.pyplot as plt
from scipy import stats

t = pd.read_csv('/kaggle/input/tah-rain-predection/train.csv')
t['Rainfall'] = t['Rainfall'].apply(transform)
t['Rainfall'].hist()

In [ ]:
from torch.utils.data import Dataset, DataLoader
class MyDataset_Train(Dataset):
 
  def __init__(self,file):
    train = pd.read_csv(file)
    train = prepare_train(train)
    y1 = train['Rainfall'].values
    y2 = train['RainTomorrow'].values
    x = train.drop(columns=['Rainfall', 'RainTomorrow']).values
    
    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y1_train=torch.tensor(np.expand_dims(y1, axis=1),dtype=torch.float32)
    self.y2_train=torch.tensor(np.expand_dims(y2, axis=1),dtype=torch.float32)
 
  def __len__(self):
    return len(self.y1_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y1_train[idx],self.y2_train[idx]

In [ ]:
from torch.utils.data import Dataset, DataLoader
class MyDataset_validation(Dataset):
 
  def __init__(self,file):
    train = pd.read_csv(file)
    train = prepare_validation(train)
    y1 = train['Rainfall'].values
    y2 = train['RainTomorrow'].values
    x = train.drop(columns=['Rainfall', 'RainTomorrow']).values
    
    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y1_train=torch.tensor(np.expand_dims(y1, axis=1),dtype=torch.float32)
    self.y2_train=torch.tensor(np.expand_dims(y2, axis=1),dtype=torch.float32)
 
  def __len__(self):
    return len(self.y1_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y1_train[idx],self.y2_train[idx]

In [ ]:
class MyDataset_Test(Dataset):
 
  def __init__(self,file):
    test = pd.read_csv(file)
    x = prepare_test(test).values
    
    self.x_train=torch.tensor(x,dtype=torch.float32)
 
  def __len__(self):
    return len(self.x_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader

# /kaggle/input/tah-rain-predection/train.csv
# /kaggle/input/tah-rain-predection/test.csv
train = MyDataset_Train('train.csv')#'/kaggle/input/tah-rain-predection/train.csv') 
train_loader=DataLoader(train,batch_size=10,shuffle=True) #batch_size=10
validation = MyDataset_validation('validation.csv')
validation_loader=DataLoader(validation,batch_size=10,shuffle=True)
test = MyDataset_Test('/kaggle/input/tah-rain-predection/test.csv') 
test_loader=DataLoader(test,batch_size=len(test),shuffle=False)

num_feature = len(next(iter(train))[0])
class LinearRegression(torch.nn.Module):

    def __init__(self): 
          super(LinearRegression, self).__init__() 
          self.linear1 = torch.nn.Linear(num_feature, 256)
          self.relu1 = torch.nn.ReLU()
          self.linear2 = torch.nn.Linear(256, 256)
          self.relu2 = torch.nn.ReLU()
#           self.dropout = torch.nn.Dropout(0.3)
          self.linear3 = torch.nn.Linear(256, 1)
          self.relu3 = torch.nn.ReLU()
          self.linear4 = torch.nn.Linear(256, 1)
      
    def forward(self, x): 
               out = self.linear1(x)
               out = self.relu1(out)
               out = self.linear2(out)
               out = self.relu2(out)
               #out = self.dropout(out)
               predict_y1 = self.linear3(out) #abs(self.linear3(out)) 
               predict_y1 = self.relu3(predict_y1)
               predict_y2 = self.linear4(out)
               return predict_y1 ,predict_y2
linear_model = LinearRegression().to('cpu')

In [ ]:
# linear_model = LinearRegression().to('cpu')
linear_model.train()
define_criterion1 = torch.nn.MSELoss()#size_average=False)
define_criterion2 = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([38947/11053]))
SGD_optimizer = torch.optim.Adam(linear_model.parameters(), lr=0.000001)#0.0001)
batch_size = 10
for epoch in range(2): 
    loss = 0
    
    for feature, label1,label2 in train_loader:
        SGD_optimizer.zero_grad() 
        predict_y1, predict_y2= linear_model(feature) 
        loss1 = define_criterion1(predict_y1, label1)
        loss2 = define_criterion2(predict_y2, label2)
        loss = loss1 + loss2
        loss.backward() 
        SGD_optimizer.step() 
    print('epoch {}, loss function {}'.format(epoch, loss.item()))

In [ ]:
linear_model.eval() 
# pred1,pred2=[],[]
for inp in test_loader:
    predict_y1,predict_y2 = linear_model(inp)
predict_y2 = np.where(predict_y2 >=0.5, 1, 0)
predict_y2 = predict_y2.reshape((6469))
predict_y1 = target.inverse_transform(predict_y1.detach().numpy()).reshape((6469))

In [ ]:
sub = pd.read_csv('/kaggle/input/tah-rain-predection/sample.csv')
sub['Rainfall'] = predict_y1
sub['RainTomorrow'] = predict_y2
sub['Rainfall'] = target.inverse_transform(sub[['Rainfall']])
sub['Rainfall'] = abs(sub['Rainfall'].apply(inverse))
l= sub['Rainfall']==np.inf
count =0
for i in range(len(l)) :
    if(l[i]):
        count+=1
        sub.loc[i,'Rainfall']=0
count,len(sub['Rainfall'])
sub.to_csv("result.csv",index=False)

In [ ]:
sub[['RainTomorrow']].value_counts()

In [ ]:
sub[['Rainfall']]

In [ ]:
loss = 0
prediction = 0
loss_classification =0
count_1=0
count_0 = 0
count_validation = 0
for inp,label1,label2 in validation:
    predict_y1,predict_y2 = linear_model(inp)
    for predLabel,trueLabel in zip(predict_y1,label1):
        loss += (predLabel-trueLabel)**2
    pred = np.where(predict_y2 >=0.5, 1.0, 0.0).reshape((len(label2)))
    label = label2.detach().numpy().reshape((len(label2)))
    for predLabel,trueLabel in zip(pred,label):
        if(predLabel==trueLabel):
            prediction+=1
        if(predLabel==trueLabel==1):
            count_1+=1
        if(predLabel==trueLabel==0):
            count_0+=1
        loss_classification+=(predLabel-trueLabel)**2
mcrmse = ((loss.detach().numpy()/len(validation))**0.5 + (loss_classification/len(validation))**0.5)/2
print(mcrmse)
print(prediction/len(validation)) #validation_size

In [ ]:
count_0, count_1, loss_classification